In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%%sh

curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

curl https://packages.microsoft.com/config/ubuntu/22.04/prod.list > /etc/apt/sources.list.d/mssql-release.list

sudo apt-get update

sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

OK
Get:1 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main armhf Packages [15.5 kB]
Get:5 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main all Packages [1,243 B]
Get:6 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main arm64 Packages [40.0 kB]
Get:7 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main amd64 Packages [172 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0   7657      0 --:--:-- --:--:-- --:--:--  7679
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0   1119      0 --:--:-- --:--:-- --:--:--  1128
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide

In [9]:
import requests
import datetime
!pip install pyodbc
import pyodbc
import holidays
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
#Informações para conectar-se ao banco

SERVER   = 'hostnoa.database.windows.net'
DATABASE = 'dbnoa'
USERNAME = 'template'
PASSWORD = 'template'

In [21]:
#Estabelecendo conexão

connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};ENCRYPT={"yes"}'

conn = pyodbc.connect(connectionString)

In [ ]:
#Requisitando informações da API

# link do open_weather: https://openweathermap.org/
# https://openweathermap.org/api/one-call-3

API_KEY = "5643ad2c732eb521a8e7cd2723c3369c"
cidade = "são vicente"
link = f"https://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={API_KEY}&lang=pt_br"

requisitions = requests.get(link)
informations = requisitions.json()

In [ ]:
#Traduzindo o dia da semana

def get_weekday(time_info):
  #currentTime = datetime.now().strftime("%H:%M:%S")
  weekDay = datetime.date.today().weekday()

  day = ["Segunda-feira", "Terça-feira", "Quarta-feira", "Quinta-feira", "Sexta-feira", "Sabado", "Domingo"]

  return day[weekDay]

#Traduzindo o mês

def get_month(time_info):

  month = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]

  return month[time_info.month]

# Pegando a data (chave primária)

def get_date(time_info):
  return str(time_info.year) + '-' + str(time_info.month) + '-' + str(time_info.day)

#Hoje é feriado?

def get_holiday(time_info):
  feriado = holidays.country_holidays("BR", subdiv="SP")

  feriados_ano = feriado[(str(time_info.year) + '-' + "1" + '-' "1"): (str(time_info.year) + '-' + "12" + '-' "31")]
  data = str(time_info.year) + '-' + str(time_info.month) + '-' + str(time_info.day)
  for feriado in feriados_ano:
    if(data == feriado):
      return True
  return False

In [ ]:
#montando as camadas
class ResidualLayer(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, change_filters=False, SE=False, k=4, **kwargs):
        super(ResidualLayer, self).__init__(**kwargs)

        self.summ_activation = tf.keras.layers.Activation('swish')

        self.main_layers = [tf.keras.layers.GroupNormalization(groups=32),
                            tf.keras.layers.Activation('swish'),
                            tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False),
                            tf.keras.layers.Conv2D(filters*4, kernel_size=3, padding='same', use_bias=False, strides=strides),
                            tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False),
                            tf.keras.layers.GroupNormalization(groups=32),
                            tf.keras.layers.Activation('swish'),
                            tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False),
                            tf.keras.layers.Conv2D(filters*4, kernel_size=3, padding='same', use_bias=False),
                            tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False),
                            tf.keras.layers.Activation('swish')]

        if SE:
            self.queeze_block = [tf.keras.layers.GlobalAveragePooling2D(),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(filters//k),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.Activation('swish'),
                                 tf.keras.layers.Dense(filters, activation='sigmoid'),
                                 tf.keras.layers.Reshape([1, 1, filters])]
            self.SE = True
        else:
            self.SE = False

        if change_filters or strides != 1:
            self.secundary_layers = [tf.keras.layers.GroupNormalization(groups=32),
                                     tf.keras.layers.Activation('swish'),
                                     tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False, strides=strides)]
        else:
             self.secundary_layers = None

    def call(self, x):
        data = tf.identity(x)

        for layer in self.main_layers:
            data = layer(data)

        if self.SE:
            excitation_factor = tf.identity(data)
            for squeeze_layer in self.queeze_block:
                excitation_factor = squeeze_layer(excitation_factor)

            data = data*excitation_factor

        if self.secundary_layers is not None:
            for layer in self.secundary_layers:
                x = layer(x)

        return self.summ_activation(x+data)

#montando o modelo personalizado
class BinaryClassifier(tf.keras.Model):
    def __init__(self, **kwargs):
        super(BinaryClassifier, self).__init__(**kwargs)

        self.init_convolution = [tf.keras.layers.RandomRotation((0.5), value_range=(0, 1)),
                                 tf.keras.layers.RandomBrightness((0.5), value_range=(0,1)),
                                 tf.keras.layers.Conv2D(filters=64, kernel_size=6, strides=2, padding='same', activation=tf.keras.layers.Activation('swish')), #128x128x3 -> 64x64x64
                                 tf.keras.layers.BatchNormalization()]

        self.residual_layers = [ResidualLayer(filters=128, strides=2, change_filters=True),   #64x64x64  ->  32x32x256
                                ResidualLayer(filters=128, strides=2, change_filters=False),
                                ResidualLayer(filters=256, strides=2, change_filters=True),
                                ResidualLayer(filters=256, strides=2, change_filters=False)]   #8x8x512    ->  4x4x512

        self.classifier_layers = [tf.keras.layers.GlobalAveragePooling2D(), #4x4x512=8192
                                  tf.keras.layers.Dense(units=1, activation="sigmoid")] #(final probabilities)

    def call(self, x):
        for init_conv in self.init_convolution:
            x = init_conv(x)

        for residual_layer in self.residual_layers:
            x = residual_layer(x)

        for classifier_layer in self.classifier_layers:
            x = classifier_layer(x)

        return x

class MultiClassifier(tf.keras.Model):
    def __init__(self, **kwargs):
        super(MultiClassifier, self).__init__(**kwargs)

        self.init_convolution = [tf.keras.layers.Conv2D(filters=128, kernel_size=6, strides=2, padding='same', activation=tf.keras.layers.Activation('swish')), #128x128x3 -> 64x64x64
                                 tf.keras.layers.BatchNormalization()]

        self.residual_layers = [ResidualLayer(filters=256, strides=2, change_filters=True),   #64x64x64  ->  32x32x256
                                ResidualLayer(filters=256, strides=2, change_filters=False),
                                ResidualLayer(filters=512, strides=2, change_filters=True),
                                ResidualLayer(filters=512, strides=2, change_filters=False)]   #8x8x512    ->  4x4x512

        self.classifier_layers = [tf.keras.layers.GlobalAveragePooling2D(), #4x4x512=8192
                                  tf.keras.layers.Dense(units=3, activation="softmax")] #(final probabilities)

    def call(self, x):
        for init_conv in self.init_convolution:
            x = init_conv(x)

        for residual_layer in self.residual_layers:
            x = residual_layer(x)

        for classifier_layer in self.classifier_layers:
            x = classifier_layer(x)

        return x

#carregando a arquitetura treinada
custom_objects = {'ResidualLayer': ResidualLayer, 'BinaryClassifier': BinaryClassifier}
binary_predictor = tf.keras.models.load_model('/content/drive/MyDrive/NOA/computer_vision_model/binary_classifier.keras', custom_objects=custom_objects)

custom_objects = {'ResidualLayer': ResidualLayer, 'MultiClassifier': MultiClassifier}
multi_predictor = tf.keras.models.load_model("/content/drive/MyDrive/NOA/computer_vision_model/multi_classifier.keras", custom_objects=custom_objects)

In [ ]:
def prediction(image):
    image = cv2.resize(image, (128, 128))/255.0
    image = np.expand_dims(image, axis=0)

    binary_prediction = float(binary_predictor.predict(image, verbose=0)[0][0])
    if binary_prediction > 0.5:
        return 0
    else:
        multi_prediction = multi_predictor.predict(image, verbose=0)[0]
        return np.argmax(multi_prediction)+1

In [ ]:
def read_images(mp4_path):
    preds = []
    for video_file in os.listdir("/content/drive/MyDrive/NOA/original_videos/08 10 2024"):
        vidcap = cv2.VideoCapture("/content/drive/MyDrive/NOA/original_videos/08 10 2024/" + video_file)
        success, image = vidcap.read()
        count = 0
        while success:
            if count%1000 == 0:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                pred = prediction(image)
                preds.append(pred)
            success, image = vidcap.read()
            count += 1

    return sum(preds)

In [ ]:
preds = read_images('/content/drive/MyDrive/NOA/original_videos/08 10 2024') #automatizar a data aqui =D

In [ ]:
preds

803

In [24]:
time_info = datetime.datetime.now()

dados = {
    "qt_pessoas": str(preds),
    "nm_dia_semana": get_weekday(time_info),
    "nm_mes": get_month(time_info),
    "nm_clima": informations['weather'][0]['description'],
    "ic_feriado": get_holiday(time_info),
    "vl_temperatura_media": round((((round(informations['main']['temp_max']-273, 2)) + (round(informations['main']['temp_min']-273, 2)))/2),2),
    "vl_umidade": str(informations['main']['humidity']),
    "vl_velocidade_ar": str(round(informations['wind']['speed']*3.6, 2)),
    "data": get_date(time_info)
}

In [25]:
#Inserindo informações no banco

data = []

for chave,dado in dados.items():
  data.append(dado)

query = "insert into dataset(qt_pessoas, nm_dia_semana, nm_mes, nm_clima, ic_feriado, vl_temperatura_media, vl_umidade, vl_velocidade_ar, data) values (?, ?, ?, ?, ?, ?, ?, ?, ?)"

cursor = conn.cursor()

cursor.execute(query, data)
conn.commit()
cursor.close()
conn.close()